# Supercharge your Reporting Services - An essential toolkit
## Craig Porteous | [@cporteous](https://twitter.com/cporteous) | [craigporteous.com](https://craigporteous.com) | [github.com/cporteou](https://github.com/cporteou)

## Subscription Management

### Subscription Next Run Summary
---

Provides detail on the next run date for all subscriptions and the commands needed to quickly disable them.

In [1]:
USE ReportServer

SELECT
	'EXEC msdb.dbo.sp_update_job @job_name=''' + J.Name + ''',@enabled = 0' AS Disable_Cmd
	,'EXEC msdb.dbo.sp_update_job @job_name=''' + J.Name + ''',@enabled = 1' AS Enable_Cmd
	,'EXEC msdb.dbo.sp_start_job ''' + J.Name + '''' AS Run_Cmd
	,'Next Run Date' = CASE next_run_date
	WHEN 0 THEN null
	ELSE
		substring(convert(varchar(15),next_run_date),1,4) + '/' +
		substring(convert(varchar(15),next_run_date),5,2) + '/' +
		substring(convert(varchar(15),next_run_date),7,2)
	END,
	'Next Run Time' = isnull(CASE len(next_run_time)
	WHEN 3 THEN cast('00:0'
		+ Left(right(next_run_time,3),1)
		+':' + right(next_run_time,2) as char (8))
	WHEN 4 THEN cast('00:'
		+ Left(right(next_run_time,4),2)
		+':' + right(next_run_time,2) as char (8))
	WHEN 5 THEN cast('0' + Left(right(next_run_time,5),1)
		+':' + Left(right(next_run_time,4),2)
		+':' + right(next_run_time,2) as char (8))
	WHEN 6 THEN cast(Left(right(next_run_time,6),2)
		+':' + Left(right(next_run_time,4),2)
		+':' + right(next_run_time,2) as char (8))
	END,'NA')
	,Convert(XML,[ExtensionSettings]).value('(//ParameterValue/Value[../Name="TO"])[1]','nvarchar(50)') as [To]
	,Convert(XML,[ExtensionSettings]).value('(//ParameterValue/Value[../Name="RenderFormat"])[1]','nvarchar(50)') as [Render Format]
	,Convert(XML,[ExtensionSettings]).value('(//ParameterValue/Value[../Name="Subject"])[1]','nvarchar(50)') as [Subject]
	,[ModifiedDate]
	,[LastStatus]
	,[EventType]
	,[LastRunTime]
	,[DeliveryExtension]
	,[Version]
FROM 
	dbo.[Subscriptions] S 
	INNER JOIN dbo.ReportSchedule R ON S.SubscriptionID = R.SubscriptionID
	INNER JOIN msdb.dbo.sysjobs J ON Convert(nvarchar(128),R.ScheduleID) = J.name
	INNER JOIN msdb.dbo.sysjobschedules JS ON J.job_id = JS.job_id
ORDER BY next_run_date, next_run_time

### Subscription Failure Job
---
Set this job up to run daily and summarise all subscription failures in an email.  
The full script with job creation steps can be found in the Scripts folder.

In [2]:
USE ReportServer
 
DECLARE @count INT
 
SELECT
   	Cat.[Name],
   	Rep.[ScheduleId],
   	Own.[UserName],
   	ISNULL(REPLACE(Sub.[Description],'Send e-mail to ',''),' ') AS Recipients,
   	Sub.[LastStatus],
   	Cat.[Path],
	'<a href="http://ssrstoolkit/Reports/Pages/Report.aspx?ItemPath=' + Cat.[Path] + '">' + Cat.[Path] + '</a>' as [LinkedPath],
   	Sub.[LastRunTime]
FROM
   	dbo.[Subscriptions] Sub
INNER JOIN
   	dbo.[Catalog] Cat on Sub.[Report_OID] = Cat.[ItemID]
INNER JOIN
   	dbo.[ReportSchedule] Rep ON (cat.[ItemID] = Rep.[ReportID] 
	   and Sub.[SubscriptionID] =Rep.[SubscriptionID])
INNER JOIN
   	dbo.[Users] Own on Sub.[OwnerID] = Own.[UserID]
WHERE
Sub.[LastStatus] NOT LIKE '%was written%' --File Share subscription
AND Sub.[LastStatus] NOT LIKE '%pending%' --Subscription in progress. No result yet
AND Sub.[LastStatus] NOT LIKE '%mail sent%' --Mail sent successfully.
AND Sub.[LastStatus] NOT LIKE '%New Subscription%' --New Sub. Not been executed yet
AND Sub.[LastStatus] NOT LIKE '%been saved%' --File Share subscription
AND Sub.[LastStatus] NOT LIKE '% 0 errors.' --Data Driven subscription
AND Sub.[LastStatus] NOT LIKE '%succeeded%' --Success! Used in cache refreshes
AND Sub.[LastStatus] NOT LIKE '%successfully saved%' --File Share subscription
AND Sub.[LastStatus] NOT LIKE '%New Cache%' --New cache refresh plan
-- AND Sub.[LastRunTime] > GETDATE()-1

### Subscription Owner Change
---
Actually altering SSRS' tables directly is risky. This is best handled in PowerShell but...  
Remove the WHERE clause to update all subscriptions. This gives visibility in the UI of ALL subscriptions from a single account. A service account for example.

In [0]:
USE ReportServer

-- Take a quick backup
SELECT * INTO Subscriptions_backup from dbo.Subscriptions

DECLARE @OldUserID uniqueidentifier
DECLARE @NewUserID uniqueidentifier

SELECT @OldUserID = UserID FROM dbo.Users WHERE UserName = 'DOMAIN\USER' -- Old subscription owner
SELECT @NewUserID = UserID FROM dbo.Users WHERE UserName = 'DOMAIN\NEWUSER' -- New subscription owner

UPDATE dbo.Subscriptions SET OwnerID = @NewUserID WHERE OwnerID = @OldUserID
GO